In [5]:
import sys

In [6]:
sys.path.insert(0,'/global/common/cori/software/theano/0.8.2/lib/python2.7/site-packages/')
import theano
sys.path.insert(0,'/global/common/cori/software/lasagne/0.1/lib/python2.7/site-packages/')

In [7]:
#!/usr/bin/env python

"""
Usage example employing Lasagne for digit recognition using the MNIST dataset.
This example is deliberately structured as a long flat file, focusing on how
to use Lasagne, instead of focusing on writing maximally modular and reusable
code. It is used as the foundation for the introductory Lasagne tutorial:
http://lasagne.readthedocs.org/en/latest/user/tutorial.html
More in-depth examples and reproductions of paper results are maintained in
a separate repository: https://github.com/Lasagne/Recipes
"""

# from __future__ import print_function

import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne
import lasagne.layers as lay
import lasagne.objectives as obj
import lasagne.nonlinearities as non
import lasagne.updates as upd
import matplotlib
import matplotlib.pyplot as plt
import h5py
%matplotlib inline

NameError: name 'h5ls' is not defined

In [8]:
!h5ls /global/project/projectdirs/nervana/yunjie/dataset/localization/larger_hurricanes_loc.h5

/usr/bin/sh: h5ls: command not found


In [4]:
# ################## Download and prepare the MNIST dataset ##################
# This is just some way of getting the MNIST dataset from an online location
# and loading it into numpy arrays. It doesn't involve Lasagne at all.

def load_dataset():
    
#     path = '/global/project/projectdirs/nervana/yunjie/dataset/localization/larger_hurricanes_loc.h5'
#     new_loc = os.path.join(os.environ['SCRATCH'], 'hurricane_detection')
#     if not os.path.exists(new_loc):
#         h5f = h5py.File(path)
#         new_h5f = h5py.File(os.path.join(new_loc, 'hur.h5'))
#         hurs = h5f.
    # We first define a download function, supporting both Python 2 and 3.
    
#     if sys.version_info[0] == 2:
#         from urllib import urlretrieve
#     else:
#         from urllib.request import urlretrieve

#     def download(filename, source='http://yann.lecun.com/exdb/mnist/'):
#         print("Downloading %s" % filename)
#         urlretrieve(source + filename, filename)

#     # We then define functions for loading MNIST images and labels.
#     # For convenience, they also download the requested files if needed.
#     import gzip

#     def load_mnist_images(filename):
#         if not os.path.exists(filename):
#             download(filename)
#         # Read the inputs in Yann LeCun's binary format.
#         with gzip.open(filename, 'rb') as f:
#             data = np.frombuffer(f.read(), np.uint8, offset=16)
#         # The inputs are vectors now, we reshape them to monochrome 2D images,
#         # following the shape convention: (examples, channels, rows, columns)
#         data = data.reshape(-1, 1, 28, 28)
#         # The inputs come as bytes, we convert them to float32 in range [0,1].
#         # (Actually to range [0, 255/256], for compatibility to the version
#         # provided at http://deeplearning.net/data/mnist/mnist.pkl.gz.)
#         return data / np.float32(256)

#     def load_mnist_labels(filename):
#         if not os.path.exists(filename):
#             download(filename)
#         # Read the labels in Yann LeCun's binary format.
#         with gzip.open(filename, 'rb') as f:
#             data = np.frombuffer(f.read(), np.uint8, offset=8)
#         # The labels are vectors of integers now, that's exactly what we want.
#         return data

    # We can now download and read the training and test set images and labels.
    X_train = load_mnist_images('train-images-idx3-ubyte.gz')
    y_train = load_mnist_labels('train-labels-idx1-ubyte.gz')
    X_test = load_mnist_images('t10k-images-idx3-ubyte.gz')
    y_test = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

    # We reserve the last 10000 training examples for validation.
    X_train, X_val = X_train[:-10000], X_train[-10000:]
    y_train, y_val = y_train[:-10000], y_train[-10000:]

    # We just return all the arrays in order, as expected in main().
    # (It doesn't matter how we do this as long as we can read them again.)
    return X_train, y_train, X_val, y_val, X_test, y_test



In [28]:
def build_cnn(input_var=None, depth=2):
    network = lasagne.layers.InputLayer(shape=(None, 1, 28, 28), input_var=input_var)
    
    for _ in range(depth):
        network = lasagne.layers.Conv2DLayer(network, num_filters=32, filter_size=(5,5),
                                            nonlinearity=lasagne.nonlinearities.rectify,
                                            W=lasagne.init.HeNormal())
        network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2,2))
    
    network = lasagne.layers.DenseLayer(
                                lasagne.layers.dropout(network, p=0.5),
                                num_units=256,
                                nonlinearity=lasagne.nonlinearities.rectify)
    
    network = lasagne.layers.DenseLayer(network, 
                                        num_units=10,
                                        nonlinearity=lasagne.nonlinearities.softmax)
    
    return network
    

In [8]:
# ############################# Batch iterator ###############################
# This is just a simple helper function iterating over training data in
# mini-batches of a particular size, optionally in random order. It assumes
# data is available as numpy arrays. For big datasets, you could load numpy
# arrays as memory-mapped files (np.load(..., mmap_mode='r')), or write your
# own custom data iteration function. For small datasets, you can also copy
# them to GPU at once for slightly improved performance. This would involve
# several changes in the main program, though, and is not demonstrated here.
# Notice that this function returns only mini-batches of size `batchsize`.
# If the size of the data is not a multiple of `batchsize`, it will not
# return the last (remaining) mini-batch.
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0,len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx: start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]
                
                            

In [9]:
class early_stop(object):
    def __init__(self, patience=500):
        self.patience = patience   # look as this many epochs regardless
        self.patience_increase = 2  # wait this much longer when a new best is
                                      # found
        self.improvement_threshold = 0.995  # a relative improvement of this much is
                                      # considered significant
        self.validation_frequency = self.patience // 2
                                      # go through this many
                                      # minibatche before checking the network
                                      # on the validation set; in this case we
                                      # check every epoch

        self.best_validation_loss = np.inf

    def keep_training(self, val_loss, epoch):
        print epoch
        print val_loss
        print self.best_validation_loss
        if val_loss < self.best_validation_loss:
                #improve patience if loss improvement is good enough
                if val_loss < self.best_validation_loss *  \
                   self.improvement_threshold:
                    self.patience = max(self.patience, epoch * self.patience_increase)

                self.best_validation_loss = val_loss
        if self.patience <= epoch:
            return False
        else:
            return True

In [10]:
def plot_training(train_errs_or_accs, val_errs_or_accs, err_or_acc):
        plt.figure(1 if err_or_acc == 'err' else 2)
        plt.clf()
        plt.title('Train/Val ' + err_or_acc)
        plt.plot(train_errs_or_accs, label='train ' + err_or_acc)
        plt.plot(val_errs_or_accs, label='val' + err_or_acc)
        plt.legend()
        plt.show()
    

In [11]:
# Load the dataset
print("loading data...")
#X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()
datasets = load_dataset()

loading data...


In [29]:
def build_network(depth=4,width=700, load=False):
    # Prepare Theano variables for inputs and targets
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

    # Create neural network model (depending on first command line parameter)
    print("Building model and compiling functions...")

    #network = build_mlp(input_var)
    #network = build_custom_mlp(input_var, depth=depth, width=width)
    network = build_cnn(input_var)
    if load:
        with np.load('model.npz') as f:
            param_values = [f['arr_%d' % i] for i in range(len(f.files))]
            lasagne.layers.set_all_param_values(network, param_values)



    # Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
    prediction = lasagne.layers.get_output(network, deterministic=False)
    loss = obj.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    # Create update expressions for training, i.e., how to modify the
    # parameters at each training step. Here, we'll use Stochastic Gradient
    # Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
    params = lay.get_all_params(network, trainable=True)
    updates = upd.nesterov_momentum(loss, params, learning_rate=0.01, momentum=0.9)

    # Create a loss expression for validation/testing. The crucial difference
    # here is that we do a deterministic forward pass through the network,
    # disabling dropout layers.
    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                                target_var)
    test_loss = test_loss.mean()



    # As a bonus, also create an expression for the classification accuracy:
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                          dtype=theano.config.floatX)

    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_fn = theano.function([input_var, target_var], loss, updates=updates)


    # Compile a second function computing the validation loss and accuracy:
    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    return train_fn, val_fn, network

In [24]:
lasagne.init.

In [30]:
train_fn, val_fn, network = build_network()

Building model and compiling functions...


In [31]:
def train(network, train_fn, val_fn, datasets, num_epochs, save=False):
    print("Starting training...")
    X_train, y_train, X_val, y_val, X_test, y_test = datasets
    train_errs = []
    train_accs = []
    val_errs = []
    val_accs = []
    #estop = early_stop(num_epochs)
    epoch = 0
#     while True:
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_acc = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, 500, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            _, acc = val_fn(inputs, targets)
            train_acc += acc
            train_batches += 1

        # And a full pass over the validation data:
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, 500, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1

        
        train_accs.append(train_acc)
        train_errs.append(train_err)
        val_errs.append(val_err)
        val_accs.append(val_acc)
        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))
        
        
        plot_training(train_errs,val_errs, 'err')
        plot_training(train_accs,val_accs, 'acc')
        
#         if not estop.keep_training(val_err, epoch):
#             break
#         epoch += 1

    # After training, we compute and print the test error:
    test_err = 0
    test_acc = 0
    test_batches = 0
    for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        test_err += err
        test_acc += acc
        test_batches += 1
    print("Final results:")
    print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
    print("  test accuracy:\t\t{:.2f} %".format(
        test_acc / test_batches * 100))

    if save:
    # Optionally, you could now dump the network weights to a file like this:
        np.savez('model.npz', *lasagne.layers.get_all_param_values(network))
    


In [ ]:
train(network, train_fn, val_fn,datasets,5)

Starting training...
